## GLAM fit. Value based. Gamma parameter limited to 0 - 1 . (Marius Usher recommendation)

In [1]:
import glam
import pandas as pd
import numpy as np
import os.path

import matplotlib.pyplot as plt

In [2]:
import pymc3 as pm

In [3]:
np.random.seed(23) # from random.org

# 3.1. Individual GLAM estimation and out of sample prediction

## Load data

In [4]:
# Load data
sufix = '_individual_Like_NoBin_Gamma_01_NUTS_31'
data = pd.read_csv('data/FF2018_data/GlamDataFF2018_Like_NoBin_31.csv')

# Subset only necessary columns
data = data[['subject', 'trial', 'choice', 'rt',
         'item_value_0', 'item_value_1',
         'gaze_0', 'gaze_1']]
data.head()

,subject,trial,choice,rt,item_value_0,item_value_1,gaze_0,gaze_1
0,0,0,0,2009,1.10,0.95,0.568396,0.431604
1,0,1,0,3371,2.00,1.70,0.762332,0.237668
2,0,2,1,1700,1.10,2.30,0.446809,0.553191
3,0,3,1,7466,1.25,1.40,0.532352,0.467648
4,0,4,1,1889,2.00,2.30,0.529736,0.470264


## Split data in training and test sets

In [5]:
train_data = pd.DataFrame()
test_data = pd.DataFrame()

for subject in data.subject.unique():
    subject_data = data[data['subject'] == subject].copy().reset_index(drop=True)
    n_trials = len(subject_data)
    
    subject_train = subject_data.iloc[np.arange(0, n_trials, 2)].copy()
    subject_test = subject_data.iloc[np.arange(1, n_trials, 2)].copy()

    test_data = pd.concat([test_data, subject_test])
    train_data = pd.concat([train_data, subject_train])

test_data.to_csv(str('data/FF2018_data/GlamDataFF2018_preprocessed_test'+sufix+'.csv'))
train_data.to_csv(str('data/FF2018_data/GlamDataFF2018_preprocessed_train'+sufix+'.csv'))

print('Split data into training ({} trials) and test ({} trials) sets...'.format(len(train_data), len(test_data)))

Split data into training (1860 trials) and test (1860 trials) sets...


In [6]:
train_data

,subject,trial,choice,rt,item_value_0,item_value_1,gaze_0,gaze_1
0,0,0,0,2009,1.10,0.95,0.568396,0.431604
2,0,2,1,1700,1.10,2.30,0.446809,0.553191
4,0,4,1,1889,2.00,2.30,0.529736,0.470264
6,0,6,0,4259,1.85,0.65,0.482254,0.517746
8,0,8,0,2548,1.55,1.10,0.528497,0.471503
10,0,10,0,4604,1.85,1.10,0.537755,0.462245
12,0,12,0,5716,0.95,0.80,0.584666,0.415334
14,0,14,0,2447,1.85,1.10,0.586022,0.413978
16,0,16,1,6932,2.00,2.00,0.585313,0.414687
18,0,18,0,6854,1.70,1.40,0.591492,0.408508


## Individual GLAM estimation

### 1. full GLAM

In [7]:
# Fitting full GLAM
print('Fitting full GLAM individually...')

glam_full = glam.GLAM(train_data)

if not os.path.exists(str('results/estimates/glam_FF2019_full'+sufix+'.npy')):
    glam_full.make_model('individual', gamma_bounds=(0, 1), t0_val=0)
    glam_full.fit(method='NUTS', tune=1000)
else:
    print('  Found old parameter estimates in "results/estimates". Skipping estimation...')
    glam_full.estimates = np.load(str('results/estimates/glam_FF2019_full'+sufix+'.npy'))   

Fitting full GLAM individually...
Generating single subject models for 31 subjects...


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


Fitting 31 model(s) using NUTS...
  Fitting model 1 of 31...


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [tau, SNR, gamma, v]
Sampling 4 chains: 100%|██████████| 12000/12000 [00:20<00:00, 581.77draws/s]
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


  Fitting model 2 of 31...


...

  Fitting model 30 of 31...


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [tau, SNR, gamma, v]
Sampling 4 chains: 100%|██████████| 12000/12000 [00:10<00:00, 1165.99draws/s]
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


  Fitting model 31 of 31...


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [tau, SNR, gamma, v]
Sampling 4 chains: 100%|██████████| 12000/12000 [02:07<00:00, 94.36draws/s] 
The acceptance probability does not match the target. It is 0.9754605777957638, but should be close to 0.8. Try to increase the number of tuning steps.


/!\ Automatically setting parameter precision...


In [8]:
# Save parameter estimates
np.save(str('results/estimates/glam_FF2019_full'+sufix+'.npy'), glam_full.estimates)

In [9]:
pd.DataFrame.from_dict(glam_full.estimates)

,SNR,b,gamma,p_error,s,t0,tau,v
0,168.42,1.0,0.01,0.05,0.008410,[0.0],4.38,0.000045
1,44.13,1.0,0.45,0.05,0.009744,[0.0],0.57,0.000075
2,130.70,1.0,0.32,0.05,0.008012,[0.0],1.85,0.000057
3,145.60,1.0,0.03,0.05,0.007725,[0.0],4.99,0.000043
4,43.01,1.0,0.21,0.05,0.006729,[0.0],1.05,0.000107
5,131.82,1.0,0.38,0.05,0.007328,[0.0],1.32,0.000063
6,110.19,1.0,0.04,0.05,0.008844,[0.0],2.13,0.000076
7,162.41,1.0,0.07,0.05,0.008190,[0.0],2.28,0.000044
8,153.29,1.0,0.01,0.05,0.009473,[0.0],0.55,0.000053
9,159.41,1.0,0.14,0.05,0.007363,[0.0],4.92,0.000047


In [10]:
full_params = pd.DataFrame(glam_full.estimates)
full_params.to_csv(str('results/params_estimates/params_glam_FF2019_full'+sufix+'.csv'))

In [11]:
# Compute WAICs
print('Computing WAIC scores for full model...')
if not os.path.exists(str('results/waic/glam_FF2019_full'+ sufix +'.npy')):
    # Note: DIC computation does not work for ADVI fitted models
    # But we are using WAIC
    glam_full.compute_waic()
else:
    print('  Found old DIC scores in "results/waic". Skipping WAIC computation...')
    glam_full.waic = np.load(str('results/waic/glam_FF2019_full'+ sufix +'.npy'))

# Compute WAICs
np.save(str('results/waic/glam_FF2019_full'+ sufix +'.npy'), glam_full.waic)

Computing WAIC scores for full model...


/Users/pradyumna/anaconda3/lib/python3.7/site-packages/pymc3/stats.py:168: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.stack(logp)
/Users/pradyumna/anaconda3/lib/python3.7/site-packages/pymc3/stats.py:219: UserWarning: For one or more samples the posterior variance of the
        log predictive densities exceeds 0.4. This could be indication of
        WAIC starting to fail see http://arxiv.org/abs/1507.04544 for details
        
  """)
/Users/pradyumna/anaconda3/lib/python3.7/site-packages/pymc3/stats.py:219: UserWarning: For one or more samples the posterior variance of the
        log predictive densities exceeds 0.4. This could be indication of
        WAIC starting to fail see http://arxiv.org/abs/1507.04544 for details
        
  """)
/Users/pradyumna/anaconda3/lib/python3.7/site-packages/pymc3/stats.

In [12]:
waic_values = pd.DataFrame(glam_full.waic)
waic_values.to_csv(str('results/params_estimates/waic_estimate'+sufix+'.csv'))

In [13]:
glam_full.waic

array([[1.06124893e+03, 0.00000000e+00, 2.03652214e+00, 1.00000000e+00],
       [1.18963142e+03, 0.00000000e+00, 9.79994007e+01, 1.00000000e+00],
       [1.02290559e+03, 0.00000000e+00, 1.60228762e+00, 1.00000000e+00],
       [1.05220215e+03, 0.00000000e+00, 2.21889129e+00, 1.00000000e+00],
       [9.64989586e+02, 0.00000000e+00, 6.55942766e+00, 1.00000000e+00],
       [1.03514484e+03, 0.00000000e+00, 1.80632802e+00, 1.00000000e+00],
       [1.03896140e+03, 0.00000000e+00, 1.87672789e+00, 1.00000000e+00],
       [1.06510670e+03, 0.00000000e+00, 1.82869081e+00, 1.00000000e+00],
       [1.12112177e+03, 0.00000000e+00, 2.48540304e+00, 1.00000000e+00],
       [1.07670638e+03, 0.00000000e+00, 1.85854108e+00, 1.00000000e+00],
       [1.10381469e+03, 0.00000000e+00, 2.34401225e+01, 1.00000000e+00],
       [1.04153469e+03, 0.00000000e+00, 2.80140053e+00, 1.00000000e+00],
       [1.07436140e+03, 0.00000000e+00, 2.29469250e+00, 1.00000000e+00],
       [1.09054524e+03, 0.00000000e+00, 1.32672704e

In [14]:
# Compute LOO

glam_full.loo = pm.loo(trace=glam_full.trace, model=glam_full.model)
glam_full.loo
np.save(str('results/loo/glam_FF2019_full'+ sufix +'.npy'), glam_full.loo)

AttributeError: 'list' object has no attribute 'nchains'

In [15]:
glam_full.loo

AttributeError: 'GLAM' object has no attribute 'loo'

In [16]:
# Predictions
print('Predicting test set data using full GLAM...')
glam_full.exchange_data(test_data)

if not os.path.exists(str('results/predictions/glam_FF2019_full'+sufix+'.csv')):
    glam_full.predict(n_repeats=50)
    glam_full.prediction.to_csv(str('results/predictions/glam_FF2019_full'+sufix+'.csv'), index=False)
else:
    print('  Found old hierarchical full GLAM predictions in "results/predictions". Skipping prediction...')
    glam_full.prediction = pd.read_csv(str('results/predictions/glam_FF2019_full'+sufix+'.csv'))

glam_full.prediction.head()

Predicting test set data using full GLAM...
Replaced attached data (1860 trials) with new data (1860 trials)...


,subject,trial,repeat,choice,rt,item_value_0,gaze_0,item_value_1,gaze_1
0,0.0,0.0,0.0,0.0,2527.0,2.0,0.762332,1.7,0.237668
1,0.0,0.0,1.0,0.0,2662.0,2.0,0.762332,1.7,0.237668
2,0.0,0.0,2.0,0.0,1567.0,2.0,0.762332,1.7,0.237668
3,0.0,0.0,3.0,0.0,1586.0,2.0,0.762332,1.7,0.237668
4,0.0,0.0,4.0,0.0,1795.0,2.0,0.762332,1.7,0.237668


## 2. Plot fit

In [ ]:
print('Close Figure to continue...')
glam.plot_fit(test_data, [glam_full.prediction]);
#glam.plot_fit(test_data, [glam_full.prediction,glam_nobias.prediction]);

plt.show()

## [END] 